In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

config = SparkConf() \
                    .setAppName('Connect to Relational databases') \
                    .set("spark.jars.packages", "mysql:mysql-connector-java:8.0.33,org.postgresql:postgresql:42.6.0")

sc = SparkContext(conf=config)
spark = SparkSession(sc)

sc.setLogLevel("ERROR")
spark

2023-08-31 01:46:27,305 WARN util.Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 172.18.0.3 instead (on interface eth0)
2023-08-31 01:46:27,306 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
mysql#mysql-connector-java added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-67b1b491-14c6-4f83-ba6c-6efa55715028;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark-3.4.1-bin-without-hadoop/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 103ms :: artifacts dl 5ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.postgresql#postgresql;42.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   5   |   0   |   0   |

# MySQL

In [2]:
# Setup the JDBC connection
jdbc_url = "jdbc:mysql://mysql:3306/sakila"
connection_properties = {
      "user" : "root",
      "password" : "root",
      "driver" : "com.mysql.cj.jdbc.Driver"
    }

# Create a query
query = """
            SELECT * FROM actor
        """

# run the query
bh_bairros_df = spark.read \
                     .jdbc(url=jdbc_url, 
                           table=f"({query}) AS t", 
                           properties=connection_properties)

bh_bairros_df.show()
print(bh_bairros_df.count())
bh_bairros_df.printSchema()

2023-08-31 01:47:59,870 INFO codegen.CodeGenerator: Code generated in 123.672084 ms
2023-08-31 01:47:59,904 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-08-31 01:47:59,914 INFO scheduler.DAGScheduler: Got job 0 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-08-31 01:47:59,914 INFO scheduler.DAGScheduler: Final stage: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0)
2023-08-31 01:47:59,914 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-08-31 01:47:59,915 INFO scheduler.DAGScheduler: Missing parents: List()
2023-08-31 01:47:59,917 INFO scheduler.DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[2] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-08-31 01:47:59,983 INFO memory.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 12.7 KiB, free 434.4 MiB)
2023-08-31 01:48:00,012 INFO memory.MemoryStore: Block broadcast_0_

+--------+----------+------------+-------------------+
|actor_id|first_name|   last_name|        last_update|
+--------+----------+------------+-------------------+
|       1|  PENELOPE|     GUINESS|2006-02-15 04:34:33|
|       2|      NICK|    WAHLBERG|2006-02-15 04:34:33|
|       3|        ED|       CHASE|2006-02-15 04:34:33|
|       4|  JENNIFER|       DAVIS|2006-02-15 04:34:33|
|       5|    JOHNNY|LOLLOBRIGIDA|2006-02-15 04:34:33|
|       6|     BETTE|   NICHOLSON|2006-02-15 04:34:33|
|       7|     GRACE|      MOSTEL|2006-02-15 04:34:33|
|       8|   MATTHEW|   JOHANSSON|2006-02-15 04:34:33|
|       9|       JOE|       SWANK|2006-02-15 04:34:33|
|      10| CHRISTIAN|       GABLE|2006-02-15 04:34:33|
|      11|      ZERO|        CAGE|2006-02-15 04:34:33|
|      12|      KARL|       BERRY|2006-02-15 04:34:33|
|      13|       UMA|        WOOD|2006-02-15 04:34:33|
|      14|    VIVIEN|      BERGEN|2006-02-15 04:34:33|
|      15|      CUBA|     OLIVIER|2006-02-15 04:34:33|
|      16|

2023-08-31 01:48:01,336 INFO memory.MemoryStore: Block broadcast_1 stored as values in memory (estimated size 13.5 KiB, free 434.4 MiB)
2023-08-31 01:48:01,339 INFO memory.MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 434.4 MiB)
2023-08-31 01:48:01,339 INFO storage.BlockManagerInfo: Added broadcast_1_piece0 in memory on localhost:38259 (size: 7.1 KiB, free: 434.4 MiB)
2023-08-31 01:48:01,340 INFO spark.SparkContext: Created broadcast 1 from broadcast at DAGScheduler.scala:1535
2023-08-31 01:48:01,341 INFO scheduler.DAGScheduler: Submitting 1 missing tasks from ShuffleMapStage 1 (MapPartitionsRDD[5] at count at NativeMethodAccessorImpl.java:0) (first 15 tasks are for partitions Vector(0))
2023-08-31 01:48:01,341 INFO cluster.YarnScheduler: Adding task set 1.0 with 1 tasks resource profile 0
2023-08-31 01:48:01,343 INFO scheduler.TaskSetManager: Starting task 0.0 in stage 1.0 (TID 1) (localhost, executor 1, partition 0, PROCESS_LOCAL, 7209 

root
 |-- actor_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



2023-08-31 01:48:02,104 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 3.0 (TID 2) in 578 ms on localhost (executor 2) (1/1)
2023-08-31 01:48:02,104 INFO cluster.YarnScheduler: Removed TaskSet 3.0, whose tasks have all completed, from pool 
2023-08-31 01:48:02,105 INFO scheduler.DAGScheduler: ResultStage 3 (count at NativeMethodAccessorImpl.java:0) finished in 0.587 s
2023-08-31 01:48:02,105 INFO scheduler.DAGScheduler: Job 2 is finished. Cancelling potential speculative or zombie tasks for this job
2023-08-31 01:48:02,105 INFO cluster.YarnScheduler: Killing all running tasks in stage 3: Stage finished
2023-08-31 01:48:02,106 INFO scheduler.DAGScheduler: Job 2 finished: count at NativeMethodAccessorImpl.java:0, took 0.594104 s
2023-08-31 01:48:13,073 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on localhost:38259 in memory (size: 7.1 KiB, free: 434.4 MiB)
2023-08-31 01:48:13,074 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on localhost:41785 in memo

In [3]:
from pyspark.sql import functions as F

bh_bairros_df.write \
             .mode("overwrite") \
             .option("truncate", "true") \
             .jdbc(url=jdbc_url, 
                   table="new_table",
                   properties=connection_properties)

bh_bairros_df.printSchema()

2023-08-31 01:48:24,393 INFO codegen.CodeGenerator: Code generated in 8.297542 ms
2023-08-31 01:48:24,448 INFO spark.SparkContext: Starting job: jdbc at NativeMethodAccessorImpl.java:0
2023-08-31 01:48:24,449 INFO scheduler.DAGScheduler: Got job 3 (jdbc at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-08-31 01:48:24,449 INFO scheduler.DAGScheduler: Final stage: ResultStage 4 (jdbc at NativeMethodAccessorImpl.java:0)
2023-08-31 01:48:24,449 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-08-31 01:48:24,449 INFO scheduler.DAGScheduler: Missing parents: List()
2023-08-31 01:48:24,451 INFO scheduler.DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[13] at jdbc at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-08-31 01:48:24,471 INFO memory.MemoryStore: Block broadcast_3 stored as values in memory (estimated size 30.0 KiB, free 434.4 MiB)
2023-08-31 01:48:24,474 INFO memory.MemoryStore: Block broadcast_3_piece0 stored as bytes in

root
 |-- actor_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



2023-08-31 01:48:25,495 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 3) in 1017 ms on localhost (executor 1) (1/1)
2023-08-31 01:48:25,495 INFO cluster.YarnScheduler: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2023-08-31 01:48:25,496 INFO scheduler.DAGScheduler: ResultStage 4 (jdbc at NativeMethodAccessorImpl.java:0) finished in 1.044 s
2023-08-31 01:48:25,496 INFO scheduler.DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
2023-08-31 01:48:25,496 INFO cluster.YarnScheduler: Killing all running tasks in stage 4: Stage finished
2023-08-31 01:48:25,496 INFO scheduler.DAGScheduler: Job 3 finished: jdbc at NativeMethodAccessorImpl.java:0, took 1.048105 s


# Postgres

In [5]:
# Setup the JDBC connection
jdbc_url = "jdbc:postgresql://postgres:5432/mydb"
connection_properties = {
      "user" : "user",
      "password" : "pass",
      "driver" : "org.postgresql.Driver"
    }

# Create a query
query = """
            SELECT * FROM products
        """

# run the query
df = spark.read \
                     .jdbc(url=jdbc_url, 
                           table=f"({query}) AS t", 
                           properties=connection_properties)

df.show()
print(df.count())
df.printSchema()

2023-08-31 01:52:46,495 INFO codegen.CodeGenerator: Code generated in 13.945333 ms
2023-08-31 01:52:46,504 INFO spark.SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
2023-08-31 01:52:46,505 INFO scheduler.DAGScheduler: Got job 4 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-08-31 01:52:46,505 INFO scheduler.DAGScheduler: Final stage: ResultStage 5 (showString at NativeMethodAccessorImpl.java:0)
2023-08-31 01:52:46,505 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-08-31 01:52:46,505 INFO scheduler.DAGScheduler: Missing parents: List()
2023-08-31 01:52:46,506 INFO scheduler.DAGScheduler: Submitting ResultStage 5 (MapPartitionsRDD[16] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-08-31 01:52:46,516 INFO memory.MemoryStore: Block broadcast_4 stored as values in memory (estimated size 12.8 KiB, free 434.3 MiB)
2023-08-31 01:52:46,519 INFO memory.MemoryStore: Block broadcast_4_

+--------------------+------------------+--------------------+------+--------------------+
|                  id|              name|              amount| price|       id_categories|
+--------------------+------------------+--------------------+------+--------------------+
|1.000000000000000000|         Lampshade|100.0000000000000...|800.00|4.000000000000000000|
|2.000000000000000000|Table for painting|1000.000000000000...|560.00|9.000000000000000000|
|3.000000000000000000|     Notebook desk|10000.00000000000...| 25.50|9.000000000000000000|
|4.000000000000000000|     Computer desk|350.0000000000000...|320.50|6.000000000000000000|
|5.000000000000000000|             Chair|3000.000000000000...|210.64|9.000000000000000000|
|6.000000000000000000|        Home alarm|750.0000000000000...|460.00|4.000000000000000000|
+--------------------+------------------+--------------------+------+--------------------+

6
root
 |-- id: decimal(38,18) (nullable = true)
 |-- name: string (nullable = true)
 |--

2023-08-31 01:52:47,161 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 6.0 (TID 5) in 362 ms on localhost (executor 2) (1/1)
2023-08-31 01:52:47,161 INFO cluster.YarnScheduler: Removed TaskSet 6.0, whose tasks have all completed, from pool 
2023-08-31 01:52:47,162 INFO scheduler.DAGScheduler: ShuffleMapStage 6 (count at NativeMethodAccessorImpl.java:0) finished in 0.368 s
2023-08-31 01:52:47,162 INFO scheduler.DAGScheduler: looking for newly runnable stages
2023-08-31 01:52:47,162 INFO scheduler.DAGScheduler: running: Set()
2023-08-31 01:52:47,162 INFO scheduler.DAGScheduler: waiting: Set()
2023-08-31 01:52:47,162 INFO scheduler.DAGScheduler: failed: Set()
2023-08-31 01:52:47,177 INFO spark.SparkContext: Starting job: count at NativeMethodAccessorImpl.java:0
2023-08-31 01:52:47,178 INFO scheduler.DAGScheduler: Got job 6 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-08-31 01:52:47,178 INFO scheduler.DAGScheduler: Final stage: ResultStage 8 (count at

In [6]:
from pyspark.sql import functions as F

df.write \
     .mode("overwrite") \
     .option("truncate", "true") \
     .jdbc(url=jdbc_url, 
           table="new_table",
           properties=connection_properties)

df.printSchema()

2023-08-31 01:59:27,611 INFO codegen.CodeGenerator: Code generated in 11.538208 ms
2023-08-31 01:59:27,628 INFO spark.SparkContext: Starting job: jdbc at NativeMethodAccessorImpl.java:0
2023-08-31 01:59:27,629 INFO scheduler.DAGScheduler: Got job 7 (jdbc at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-08-31 01:59:27,629 INFO scheduler.DAGScheduler: Final stage: ResultStage 9 (jdbc at NativeMethodAccessorImpl.java:0)
2023-08-31 01:59:27,630 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-08-31 01:59:27,630 INFO scheduler.DAGScheduler: Missing parents: List()
2023-08-31 01:59:27,631 INFO scheduler.DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[27] at jdbc at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-08-31 01:59:27,643 INFO memory.MemoryStore: Block broadcast_7 stored as values in memory (estimated size 30.1 KiB, free 434.4 MiB)
2023-08-31 01:59:27,646 INFO memory.MemoryStore: Block broadcast_7_piece0 stored as bytes i

root
 |-- id: decimal(38,18) (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: decimal(38,18) (nullable = true)
 |-- price: decimal(7,2) (nullable = true)
 |-- id_categories: decimal(38,18) (nullable = true)



2023-08-31 01:59:27,845 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 9.0 (TID 7) in 194 ms on localhost (executor 1) (1/1)
2023-08-31 01:59:27,845 INFO cluster.YarnScheduler: Removed TaskSet 9.0, whose tasks have all completed, from pool 
2023-08-31 01:59:27,845 INFO scheduler.DAGScheduler: ResultStage 9 (jdbc at NativeMethodAccessorImpl.java:0) finished in 0.214 s
2023-08-31 01:59:27,846 INFO scheduler.DAGScheduler: Job 7 is finished. Cancelling potential speculative or zombie tasks for this job
2023-08-31 01:59:27,846 INFO cluster.YarnScheduler: Killing all running tasks in stage 9: Stage finished
2023-08-31 01:59:27,846 INFO scheduler.DAGScheduler: Job 7 finished: jdbc at NativeMethodAccessorImpl.java:0, took 0.217860 s
